In [198]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [132]:
boston_data=load_boston()

In [133]:
data=pd.DataFrame(boston_data.data,columns=boston_data.feature_names)

In [134]:
target=pd.DataFrame(boston_data.target)

In [135]:
final_boston=pd.concat([data,target],axis=1)

In [136]:
final_boston.rename(columns={0:'TARGET'},inplace=True)

In [137]:
final_boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TARGET
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [161]:
train, test = train_test_split(final_boston, test_size=0.2)

In [162]:
train

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TARGET
304,0.05515,33.0,2.18,0.0,0.472,7.236,41.1,4.0220,7.0,222.0,18.4,393.68,6.93,36.1
247,0.19657,22.0,5.86,0.0,0.431,6.226,79.2,8.0555,7.0,330.0,19.1,376.14,10.15,20.5
110,0.10793,0.0,8.56,0.0,0.520,6.195,54.4,2.7778,5.0,384.0,20.9,393.49,13.00,21.7
187,0.07875,45.0,3.44,0.0,0.437,6.782,41.1,3.7886,5.0,398.0,15.2,393.87,6.68,32.0
228,0.29819,0.0,6.20,0.0,0.504,7.686,17.0,3.3751,8.0,307.0,17.4,377.51,3.92,46.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.06664,0.0,4.05,0.0,0.510,6.546,33.1,3.1323,5.0,296.0,16.6,390.96,5.33,29.4
37,0.08014,0.0,5.96,0.0,0.499,5.850,41.5,3.9342,5.0,279.0,19.2,396.90,8.77,21.0
121,0.07165,0.0,25.65,0.0,0.581,6.004,84.1,2.1974,2.0,188.0,19.1,377.67,14.27,20.3
233,0.33147,0.0,6.20,0.0,0.507,8.247,70.4,3.6519,8.0,307.0,17.4,378.95,3.95,48.3


In [163]:
features = train.drop(columns='TARGET').columns

In [164]:
x_train = np.array(train.drop(columns='TARGET').values)
y_train = np.array(train['TARGET'].values)

In [165]:
x_test = np.array(test.drop(columns='TARGET').values)
y_test = np.array(test['TARGET'].values)

In [167]:
normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
normalizer.adapt(x_train)

In [192]:
tmp = normalizer(x_train)

In [199]:
print(tmp[0])
print(x_train[0])

tf.Tensor(
[-0.446  0.906 -1.298 -0.262 -0.715  1.416 -0.981  0.128 -0.282 -1.111
 -0.017  0.421 -0.805], shape=(13,), dtype=float32)
[  0.055  33.      2.18    0.      0.472   7.236  41.1     4.022   7.
 222.     18.4   393.68    6.93 ]


In [168]:
inputs = tf.keras.Input(shape=x_train.shape[1:])

In [169]:
x = normalizer(inputs)
x = tf.keras.layers.Dense(200,activation='relu') (x)
x = tf.keras.layers.Dense(100,activation='relu') (x)
x = tf.keras.layers.Dropout(0.25) (x)
x = tf.keras.layers.Dense(50,activation='relu') (x)
x = tf.keras.layers.Dense(25,activation='relu') (x)
output = tf.keras.layers.Dense(1) (x)

In [170]:
model = tf.keras.Model(inputs,output)

In [171]:
model.summary()

Model: "functional_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
normalization_8 (Normalizati (None, 13)                27        
_________________________________________________________________
dense_60 (Dense)             (None, 200)               2800      
_________________________________________________________________
dense_61 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_12 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_62 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_63 (Dense)             (None, 25)              

In [172]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [177]:
model.fit(x_train,y_train,epochs=500,verbose=0)

In [178]:
model.evaluate(x_test,y_test)

4/4 [==============================] - 0s 2ms/step - loss: 17.3414


17.34137535095215

In [188]:
print(np.array(x_test[42]))

[2.0090e-02 9.5000e+01 2.6800e+00 0.0000e+00 4.1610e-01 8.0340e+00
 3.1900e+01 5.1180e+00 4.0000e+00 2.2400e+02 1.4700e+01 3.9055e+02
 2.8800e+00]


In [189]:
model.predict(np.array(x_test[42]).reshape(-1,13))

array([[44.84079]], dtype=float32)

In [190]:
y_test[42]

50.0

In [182]:
model2 = tf.keras.Sequential([
                            tf.keras.Input(shape=input_shape),
                             tf.keras.layers.Dense(200,activation='relu'),
                             tf.keras.layers.Dense(100,activation='relu'),
                             tf.keras.layers.Dropout(0.25),
                             tf.keras.layers.Dense(50,activation='relu'),
                             tf.keras.layers.Dense(25,activation='relu'),
                             tf.keras.layers.Dense(1)
])

In [183]:
model2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 200)               2800      
_________________________________________________________________
dense_66 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_13 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_67 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_68 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_69 (Dense)             (None, 1)                 26        
Total params: 29,251
Trainable params: 29,251
Non-trainable params: 0
__________________________________________________

In [184]:
model2.compile(optimizer='adam',loss='mean_squared_error')

In [185]:
model2.fit(x_train,y_train,epochs=500,verbose=0)

In [186]:
model2.evaluate(x_test,y_test)

4/4 [==============================] - 0s 2ms/step - loss: 65.3252


65.32518768310547

In [191]:
model2.predict(np.array(x_test[42]).reshape(-1,13))

array([[35.420166]], dtype=float32)